In this notebook it will be explained how to get and plot the data from the USGS Earthquake Catalog. 

The USGS Earthquake Catalog is a database of earthquakes that have occurred around the world. Using the library "libcomcat" we can acces the data and work in our study, which is to analyze the peligrosity of the earthquakes in an arbitrary area of interest. 

For our repository we have decided so use the next convention: "the first letter shows the level of the folders being "A" for the first level and "B" for the second level. Then, the numbers that follows the letter are used to organize the folders as we pleased. 

The scripts and folders regarded to the seismic events will have an "eq_" prefix.



The scripts are organized in the following way:

-A00_data: This folder will be the one in charge to the download and storage of the data. For now, it only serves as a temporary storage, the database will eventually be stored in a more permanent way.

-A01_scripts: This folder will contain the scripts that will be used to download and process the data. Here we can find the folder "B01_eq_download", where the script "eq_download.py" is located and the utils tha it uses. This script will download the data from the USGS Earthquake Catalog and store it in the folder "A00_data". There is also the directory "B01_4_eq_processing" where we have "preprocess.py" and "process.py". These scripts are the responsible to merge both data frames obtained previously and process the data.

-A02_utils: here the general utils are stored, those that can be used for any script or that are going to be used to create the dashboard.

-A03_performance:

-A04_web: in this directory is the Quarto script that creates the dashboard.

-A05_config:

-A06_tests:

-A07_docs: 

-A08_notebooks: where notebooks are stored.

# Functions:


## Functions from B01_2_eq_dowloand

### eq_download.py

    -search_by_min_magnitude(date_i, date_f, min_magn, center_coords, reg_rad)

This function calls the libcomcat functions search(), get_summary_data_frame() and get_detail_data_frame(), to get the basic information of the events and their magnitudes information. 

Its parameters are: starting date (date_i), ending date (date_f), minimum magnitude of the events (min_mag), coordinates of the volcano we want to study (center_coords in form (latitude, longitude) list) and the radius of the area we want to study in km (reg_rad).

The function returns a dataframe with the parameters that will be used to get the trigger index by merging the two dataframes obtained from the previous functions with the auxiliary function working_df().

    -download_all_by_region(date_i, date_f, center_coords, reg_rad)

This is the main function for the script since it requests to the USGS catalog the data for all earthquakes in the region with a minimum magnitude of 0.001, since we can be also interested in looking for the magnitude of completeness. 

It uses the same parameters as the previous function, but it does not require the minimum magnitude. The return is the same dataframe, but with an wider list of events.
    
    -working_df(df1, df2)

As it is said before, this is an auxiliary function that is used to merge the two dataframes obtained from the libcomcat functions. It is necessary because the we can not know which type of magnitude is the function get_summary_data_frame() going to return. Thus, with this function we can merge, by the ID of the events, the two dataframes and get the informatio1n we need.

### utils.py

    -get_magnitude_type(mag_type)